# Modeling

In [33]:
import pandas as pd
import numpy as np 

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

import random
random.seed(222)

In [50]:
X_train_l = pd.read_csv('./data/listings_Xtrain.csv', index_col='Unnamed: 0')
X_test_l = pd.read_csv('./data/listings_Xtest.csv', index_col='Unnamed: 0')

X_train_r = pd.read_csv('./data/X_train_r.csv', index_col='Unnamed: 0')
X_test_r = pd.read_csv('./data/X_test_r.csv', index_col='Unnamed: 0')


y_train_r = pd.read_csv('./data/y_train_r.csv', index_col='Unnamed: 0')
y_test_r = pd.read_csv('./data/y_test_r.csv', index_col='Unnamed: 0')

# Listings Regression Modeling

## Linear Regression

In [3]:
lr = LinearRegression()

In [29]:
lr.fit(X_train_l, y_train)

LinearRegression()

In [30]:
lr.score(X_train_l, y_train)

0.07829814051081252

In [31]:
lr.score(X_test_l, y_test)

-0.04302994494945511

## K Neighbors Regressor

## Decision Tree Regressor

## Random Forests Regressor

## AdaBoost Regressor

## GradientBoost Regressor

## Bagging Regressor

# Reviews Regression Modeling

## Linear Regression

In [45]:
y_train_r = y_train_r['review_scores_rating']

In [46]:
X_train_r.shape, y_train_r.shape

((14000, 1), (14000,))

In [51]:
y_train_r.head()

,review_scores_rating
61715,4.84
214404,4.79
314945,4.91
88069,4.85
493922,4.95


In [57]:
pls_work = X_train_r.join(y_train_r)

In [59]:
pls_work.shape

(14000, 2)

In [66]:
X = pls_work['lem_comments']
y = pls_work['review_scores_rating']

In [67]:
X.shape, y.shape

((14000,), (14000,))

In [68]:
y

61715     4.84
214404    4.79
314945    4.91
88069     4.85
493922    4.95
          ... 
431201    4.36
103342    4.91
146289    4.93
337558    4.94
337477    4.94
Name: review_scores_rating, Length: 14000, dtype: float64

In [71]:
pls_work.isnull().sum()

lem_comments            101
review_scores_rating      0
dtype: int64

In [69]:
pipe_cvec_lr = Pipeline([
    ('cvec', CountVectorizer()),
    ('lr', LinearRegression())
])

pipe_cvec_lr_params = {
    'cvec__stop_words': ['english', None],
    'cvec__ngram_range': [(1,1), (1,2)] 
}

gs_cvec_lr = GridSearchCV(pipe_cvec_lr, pipe_cvec_lr_params, n_jobs = -1)
gs_cvec_lr.fit(X, y)

ValueError: 
All the 20 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/myacarrizosa/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/myacarrizosa/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 378, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/Users/myacarrizosa/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 336, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/Users/myacarrizosa/opt/anaconda3/lib/python3.9/site-packages/joblib/memory.py", line 349, in __call__
    return self.func(*args, *